In [11]:
import random
import numpy as np
import heapq
from tqdm import tqdm
import copy

In [1]:
class player():
    """Just to store information"""
    def __init__(self, chip = 11 ):
        self.chip = chip
        self.card = []
    
    def __repr__(self):
        return f'Chip: {self.chip} | Card owned: {self.card}'
    
    def action(self, move):
        if move == 'take':
            self.card.append(self.game.current_card)
            self.chip += self.game.chip_in_pot
        elif move == 'pass':
            self.chip -= 1
            self.game.chip_in_pot += 1
    
    def calculate_score(self):
        sorted_card = sorted(self.card)
        total_score = 0
        if len(sorted_card) == 0:
            return self.chip
        total_score -= sorted_card[0]
        for score, prev_score in zip(sorted_card[1:], sorted_card[:-1]):
            if score - prev_score > 1:
                total_score -= score
        total_score += self.chip
        return total_score

In [3]:
class game:
    full_deck = [i for i in range(3,36)]
    
    def __init__(self,
                 n_player: int = 3,
                 n_chip: int = 11,
                 n_remove_card: int = 9
                 ):
        self.n_remove_card = n_remove_card
        self.remain_card = game.full_deck
        self.played_card = []
        self.current_card = self.flip_card()
        
        self.chip_in_pot = 0
        self.turn = 0 # need this to know which player to take
        self.n_player = n_player
        self.n_chip = n_chip

        self.max_score = 1
        self.min_score = -1
        self.score_range = self.max_score - self.min_score
        self.rollout_rule = {'pass': 0.9,
                             'take': 0.1}
        self.is_continue = True
        self.init_player(n_player)
    
    def __len__(self):
        return len(self.remain_card)
    
    def __str__(self):
        return f'Current card: {self.current_card} | Played cards: {self.played_card}'
    def __repr__(self):
        return f'Current card: {self.current_card} | Played cards: {self.played_card}'

    def init_player(self, n_player):
        self.players = [player(chip =  self.n_chip) for _ in range(n_player)]
        
    def set_remain_card(self):
        """Update list of remaining cards"""
        self.remain_card = [i for i in game.full_deck 
                            if i not in self.played_card]

    def set_removed_card(self):
        "Remove 9 cards"
        self.removed_card = self.get_cards(9)
        self.set_remain_card()

    def get_cards(self, n):
        'Randomize a card, if number of card remained == 9'
        if len(self.remain_card) == self.n_remove_card:
            return False
        card = random.sample(self.remain_card, n)
        return card[0] if n == 1 else card
    
    
    def flip_card(self, 
                card: int = None):
        '''Progress the game by flipping a new card'''
        if card == None:
            card = self.get_cards(1)
            if not card:
                return card
        self.played_card.append(card)
        self.set_remain_card()
        self.current_card = card
        return card
    
    def calculate_ranking(self):
        score_list = np.array([player.calculate_score() for player in self.players])
        ranking = score_list.argsort()
        step = self.score_range/(self.n_player - 1)
        final_score = [self.min_score + step*rank for rank in ranking]
        return final_score
    
    def get_legal_action(self):
        if not self.is_continue:
            print('Game over, no legal action')
            return []
        current_player = self.players[self.turn]
        if current_player.chip > 0:
            return ['pass', 'take']
        else:
            return ['take']

    def next_player(self, move):
        """get next player if take then turn does not change"""
        if move == 'take':
            return self.turn
        elif move == 'pass':
            return self.turn + 1 if self.turn + 1 < self.n_player else 0
            
    def action(self, move):
        if not self.is_continue:
            print('Game over, cannot act')
            return False
        current_player = self.players[self.turn]
        if move == 'take':
            current_player.card.append(self.current_card)
            current_player.chip += self.chip_in_pot
            self.chip_in_pot = 0
            is_continue = self.flip_card()
            self.is_continue = is_continue
            return self.is_continue

        elif move == 'pass':
            current_player.chip -= 1
            self.chip_in_pot += 1
            self.turn = self.next_player(move)
            return True
    
    def rollout_policy_rule(self):
        legal_action = self.get_legal_action()
        weight = [self.rollout_rule.get(i) for i in legal_action]
        move = random.choices(legal_action, weight)[0]
        return move
    
    def rollout_policy_1(self):
        current_player = self.players[self.turn]
        legal_action = self.get_legal_action()
        self.chip_in_pot
        weight = [self.rollout_rule.get(i) for i in legal_action]
        move = random.choices(legal_action, weight)[0]
        return move


    def self_play(self):
        """Keeps playing till the game end
        This is where you set your rollout policy
        """
        while self.is_continue:
            move = self.rollout_policy_rule()
#             print(f'''Card: {self.current_card} | Chip in pot: {self.chip_in_pot} | Player: {self.turn} - {self.players[self.turn]}
# move: {move}'''
# )
            self.is_continue = self.action(move)        
        score_list = self.calculate_ranking()
        return score_list

In [4]:
# #test
# nothanks = game()
# nothanks.set_removed_card()
# nothanks.flip_card()
# print(nothanks)

# while len(nothanks.remain_card) > 0:
#     new_card = nothanks.get_cards(1)
#     nothanks.flip_card(new_card)
#     print(nothanks)


In [5]:
class game_state:
    """This class is the node of the tree"""
    def __init__(self, parent = None, game = None, depth = 0):
        self.depth = depth
        self.win = 0
        self.n_explored = 0
        self.parent = parent
        self.child = [] # (uct, move, next node)
        self.game = game
        self.turn = game.turn # need this as reference when doing backpropagation

    def calculate_uct(self):
        """Can set your uct policy here"""
        if self.n_explored == 0:
            return np.inf
        else:
            exploitation = self.win/self.n_explored
            exploration = (2*np.log(self.parent.n_explored)/self.n_explored)**0.5
            return exploitation + exploration
        
    def get_best_move(self):
        win_list = [child_node[-1].win/child_node[-1].n_explored for child_node in self.child]
        return self.child[np.argmax(win_list)][1]

In [6]:
# class operator:
#     """supervise the game, tell whose turn it is"""
#     def __init__(self, n_player: int, n_chip: int, game: object):
#         """n_chip: number of chip for each player"""
#         self.n_player = n_player
#         self.n_chip = n_chip # each player
#         self.init_player()
#         self.min_score = -1
#         self.max_score = 1
#         self.score_range = self.max_score - self.min_score

#     def get_legal_action(self, 
#                          player: object):
#         if player.chip > 0:
#             return ['pass', 'take']
#         else:
#             return ['take']
        
#     def action(self, game, move):
#         # Should make copies here? then return the copies?
#         player = game.players[game.turn]
#         if move == 'take':
#             player.card.append(game.current_card)
#             player.chip += game.chip_in_pot
#         elif move == 'pass':
#             player.chip -= 1
#             game.chip_in_pot += 1
    
#     def next_player(self, game, move):
#         """get next player if take then turn does not change"""
#         if move == 'take':
#             return game.turn
#         elif move == 'pass':
#             return game.turn + 1 if game.turn + 1 < game.n_player else 0

In [7]:
class mcts():
    def __init__(self, 
                 depth = 10, 
                 branch = 0):
        """Depth: limit on the number of card flip to look in the future"""
        self.depth = depth
        self.branch = branch
        self.simulation_iter = 100

    def get_best_uct(self, child_list):
        index_max = np.argmax([i[0] for i in child_list])
        return child_list[index_max]
    
    def simulation(self, game, iter = 100):
        score_list = np.array([0.]*game.n_player)
        for _ in range(iter):
            game_tmp = copy.deepcopy(game)
            # print(f'--------------------------------{game_tmp.is_continue} | {game_tmp}')
            
            score_list_tmp = game_tmp.self_play()
            score_list += score_list_tmp
        return score_list


    def selection(self, game_node):
        game = game_node.game
        score_list, total_explored = np.array([0.]*game.n_player), 0

        if game_node.depth > self.depth:
            return score_list, total_explored

        if len(game_node.child) == 0:
            # EXPANSION
            # if game_node.depth < self.depth: # stop expanding -> switch to simulation
            legal_move = game.get_legal_action()
            for move in legal_move:     
                game_clone = copy.deepcopy(game)
                game_clone.action(move)
                next_node = game_state(parent = game_node,
                                        game = game_clone,
                                       depth = game_node.depth + 1 if move == 'take' else game_node.depth)
                uct_tmp = next_node.calculate_uct()
                heapq.heappush(game_node.child, [uct_tmp, move, next_node])

            # SIMULATION
            total_explored = 0
            for _, _, child in game_node.child:
                score_list_tmp = self.simulation(child.game, self.simulation_iter)
                
                child.n_explored += self.simulation_iter
                child.win += score_list_tmp[child.turn]

                score_list += score_list_tmp
                total_explored += self.simulation_iter
            
            # Update the current node here too
            game_node.n_explored += total_explored
            if game_node.parent:
                game_node.win += score_list[game_node.parent.turn]
            return score_list, total_explored 

        else:
            #SELECTION
            for child in game_node.child:
                child[0] = child[-1].calculate_uct()
            
            _, _, next_node = self.get_best_uct(game_node.child)
            score_list, total_explored = self.selection(next_node)
            
            #BACKPROPAGAION
            game_node.n_explored += total_explored
            if game_node.parent:
                game_node.win += score_list[game_node.parent.turn]

            return score_list, total_explored

In [8]:
# test self play

In [124]:
a = game()

In [125]:
a.self_play()

Card: 16 | Chip in pot: 0 | Player: 0 - Chip: 11 | Card owned: []
move: pass
Card: 16 | Chip in pot: 1 | Player: 1 - Chip: 11 | Card owned: []
move: pass
Card: 16 | Chip in pot: 2 | Player: 2 - Chip: 11 | Card owned: []
move: pass
Card: 16 | Chip in pot: 3 | Player: 0 - Chip: 10 | Card owned: []
move: pass
Card: 16 | Chip in pot: 4 | Player: 1 - Chip: 10 | Card owned: []
move: pass
Card: 16 | Chip in pot: 5 | Player: 2 - Chip: 10 | Card owned: []
move: pass
Card: 16 | Chip in pot: 6 | Player: 0 - Chip: 9 | Card owned: []
move: pass
Card: 16 | Chip in pot: 7 | Player: 1 - Chip: 9 | Card owned: []
move: pass
Card: 16 | Chip in pot: 8 | Player: 2 - Chip: 9 | Card owned: []
move: pass
Card: 16 | Chip in pot: 9 | Player: 0 - Chip: 8 | Card owned: []
move: pass
Card: 16 | Chip in pot: 10 | Player: 1 - Chip: 8 | Card owned: []
move: pass
Card: 16 | Chip in pot: 11 | Player: 2 - Chip: 8 | Card owned: []
move: pass
Card: 16 | Chip in pot: 12 | Player: 0 - Chip: 7 | Card owned: []
move: pass
Car

[np.float64(1.0), np.float64(-1.0), np.float64(0.0)]

In [126]:
a.flip_card()

False

In [127]:
a.get_legal_action()

Game over, no legal action


[]

In [136]:
a.action('take')
a.players

Game over, cannot act


[Chip: 24 | Card owned: [16, 24, 23, 7, 33, 26, 29],
 Chip: 5 | Card owned: [19, 21, 10, 9, 32, 20, 15, 27],
 Chip: 4 | Card owned: [13, 28, 22, 6, 12, 3, 34, 4, 31]]

In [133]:
a.flip_card()
a.played_card

[16,
 19,
 13,
 21,
 28,
 22,
 10,
 9,
 6,
 12,
 24,
 3,
 32,
 20,
 34,
 15,
 4,
 23,
 7,
 33,
 27,
 26,
 31,
 29]

In [138]:
# test mcts

In [12]:
nothanks = game()
game_node = game_state(game = nothanks)
tree = mcts()

In [13]:
for child in game_node.child:
    print(f'move: {child[0]} | win pct: {child[-1].calculate_uct()}')

In [14]:
print(nothanks)
while nothanks.is_continue:
    for _ in tqdm(range(50)):
        tree.selection(game_node= game_node)
    move = game_node.get_best_move()
    print(f'''Card: {nothanks.current_card} | Chip in pot: {nothanks.chip_in_pot} | Player: {nothanks.turn} - {nothanks.players[nothanks.turn]}
move: {move}'''
)
    nothanks.action(move)
    game_node = game_state(game = nothanks)

Current card: 4 | Played cards: [4]


100%|██████████| 50/50 [00:06<00:00,  7.80it/s]


Card: 4 | Chip in pot: 0 | Player: 0 - Chip: 11 | Card owned: []
move: pass


100%|██████████| 50/50 [00:06<00:00,  7.88it/s]


Card: 4 | Chip in pot: 1 | Player: 1 - Chip: 11 | Card owned: []
move: pass


...

Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
G

100%|██████████| 50/50 [00:00<00:00, 1020.39it/s]


Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
G

100%|██████████| 50/50 [00:00<00:00, 2272.89it/s]

Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
Game over, no legal action
G

In [16]:
nothanks.calculate_ranking()

[1.0, 0.0, -1.0]

In [20]:
for user in nothanks.players:
    print(user, user.calculate_score())

Chip: 0 | Card owned: [4, 13, 22, 26] -65
Chip: 29 | Card owned: [10, 20, 30, 32, 29, 11, 5, 6, 28, 16, 9, 31, 19, 15, 25, 12] -72
Chip: 4 | Card owned: [3, 35, 27, 17] -78
